In [268]:
import pandas as pd
import numpy as np 
import re
import warnings
warnings.filterwarnings('ignore')

In [269]:
movie_data=pd.read_csv(r"C:\Users\Administrator\Desktop\Machine Learning project-MaktabKhooneh\IMBD\movies.csv")
rating_data=pd.read_csv(r"C:\Users\Administrator\Desktop\Machine Learning project-MaktabKhooneh\IMBD\ratings.csv")

در قسمت اول که سیستم توصیه گر مبتنی بر محتوا است ما به ژانر فیلم ها نیاز داریم تا براساس ورودی که از کاربر میگیریم بتونیم فیلم ها و ژانر اون ها رو پیدا کنیم و بر اساس بالاتین امتیازی که هر ژانری گرفته بتونیم به کاربر فیلم توصیه کنیم 
پس نیازی به دیتاست مربوط به "ریتینگ" نداریم  

In [270]:
movie_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


 لازمه که ستون های مرتبط به ژانر رو اسپارس کنیم تا بر اساس آن ببینم هر فیلمی شامل چه ژانر هایی میشود 
از طرفی ورودی کاربر بر اساس نام فیلم است پس لازمه که ستون عنوان رو بر اساس عنوان تمیز کنیم و دیتای سال رو جدا در نظر بگیریم تا بتونیم بر اساس اسم فیلمی که در پروفایل کاربر بوده نوع ژانر رو پیدا کنیم 

In [271]:
movie_data["year"]=movie_data["title"].str.extract("(\(\d+\))")

In [272]:
movie_data["title"]=movie_data["title"].str.replace("(\(\d+\))" , "" , regex=True)
movie_data["title"]=movie_data["title"].str.strip()

In [273]:
movie_data["genres"]=movie_data["genres"].str.split("|")

In [274]:
moviesGenre=movie_data.copy()

In [275]:
for index , rows in movie_data.iterrows():
    for genre in rows["genres"]:
        moviesGenre.at[index , genre]=1
moviesGenre.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",(1995),1.0,1.0,1.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Jumanji,"[Adventure, Children, Fantasy]",(1995),1.0,NaN,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Grumpier Old Men,"[Comedy, Romance]",(1995),NaN,NaN,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",(1995),NaN,NaN,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Father of the Bride Part II,[Comedy],(1995),NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [276]:
moviesGenre.fillna(0 , inplace=True)

جهت انجام ضرب ماتریسی لازمه که جاهای خالی جدول با صفر پر کنیم 

In [277]:
moviesGenre.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",(1995),1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",(1995),1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",(1995),0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",(1995),0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],(1995),0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


اطلاعات ورودی که در پروفایل کاربر وجود دارد و امتیاز هایی که به فیلم داده  

In [278]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
userInputpd=pd.DataFrame(userInput)
userInputpd

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


خب حالا اینجا به دو تا چیز نیاز داریم یکی اینکه فیلم هایی که کاربر دیده چه ژانری دارند که از طریق عنوان در دیتاست فیلم ها جستجو میکنیم و جدا کنیم و دومی هم اینکه دیتاهای به دست آمده رو با پروفایل کاربر ادغام کنیم تا یک جدول کاملی داشته باشیم که شامل آیدی و نام فیلم و رتبه فیلم هایی باشه که کاربر دیده      

In [279]:
user_movie=movie_data[movie_data["title"].isin(userInputpd["title"].tolist())]
user_movie.drop(columns=["year" , "genres"] , axis=1 , inplace=True)

In [280]:
user_movie.head()

,movieId,title
0,1,Toy Story
1,2,Jumanji
257,296,Pulp Fiction
973,1274,Akira
1445,1968,"Breakfast Club, The"


In [281]:
user_id=pd.merge(user_movie , userInputpd)
user_id

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


In [282]:
table_user=moviesGenre[moviesGenre["movieId"].isin(user_id["movieId"].tolist())]

In [283]:
table_user.drop(columns=["title" , "genres", "year"] , axis=1 , inplace=True)

In [284]:
table_user.set_index("movieId" , inplace=True)

In [285]:
table_user.reset_index(drop=True , inplace=True)

In [286]:
user_interest=table_user.transpose().dot(user_id["rating"])

در اینجا با ضرب امتیاز های فیلم هایی که کاربر دیده در ژانر های همان فیلم ها میتونیم میزان علاقه مندی کاربر به ژانر ها رو وزن دهی کنیم  که نتایج نشون میده کاربر به ژانر کمدی و درام و ماجراجویی علاقه داره  

In [287]:
user_interest

Adventure             10.0
Animation              8.0
Children               5.5
Comedy                13.5
Fantasy                5.5
Romance                0.0
Drama                 10.0
Action                 4.5
Crime                  5.0
Thriller               5.0
Horror                 0.0
Mystery                0.0
Sci-Fi                 4.5
War                    0.0
Musical                0.0
Documentary            0.0
IMAX                   0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

In [288]:
moviesGenre.drop(columns=["genres" , "movieId" , "year"] , axis=1,inplace=True)

In [289]:
moviesGenre.set_index("title" , inplace=True)

من در اینجا عنوان رو به عنوان شاخص در نظر گرفتم تا مستقیما وقتی جدول اصلی را در ضریب های علاقه مندی کاربر ضرب میکنیم مستقیم فیلم هارو نشون بده  

In [290]:
moviesGenre

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
title,,,,,,,,,,,,,,,,,,,,
Toy Story,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jumanji,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Grumpier Old Men,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Waiting to Exhale,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Father of the Bride Part II,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Black Butler: Book of the Atlantic,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
No Game No Life: Zero,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Flint,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [291]:
recommend_table=((moviesGenre *user_interest ).sum(axis=1))/(user_interest.sum())

In [292]:
recommend_table.sort_values(ascending=False).head()

title
Inside Out                             0.734266
Wizards of Waverly Place: The Movie    0.685315
Dragonheart 2: A New Beginning         0.678322
Interstate 60                          0.678322
Rubber                                 0.671329
dtype: float64

نتایج حاصل از این حدول نشون میده که فیلم هایی که در الویت قرار دارند تا به کاربر پیشنهاد بدیم و همانطور که  مشاهده میکنید حتما ژانر کمدی و درام و ماجراجویی در ژانر این فیلم ها هست 

In [293]:
movie_data.loc[movie_data["title"].isin(recommend_table.sort_values(ascending=False).head(10).keys())]

,movieId,title,genres,year
1390,1907,Mulan,"[Adventure, Animation, Children, Comedy, Drama...",(1998)
2250,2987,Who Framed Roger Rabbit?,"[Adventure, Animation, Children, Comedy, Crime...",(1988)
4631,6902,Interstate 60,"[Adventure, Comedy, Drama, Fantasy, Mystery, S...",(2002)
5490,26340,"Twelve Tasks of Asterix, The (Les douze travau...","[Action, Adventure, Animation, Children, Comed...",(1976)
7441,81132,Rubber,"[Action, Adventure, Comedy, Crime, Drama, Film...",(2010)
8349,108540,Ernest & Célestine (Ernest et Célestine),"[Adventure, Animation, Children, Comedy, Drama...",(2012)
8357,108932,The Lego Movie,"[Action, Adventure, Animation, Children, Comed...",(2014)
8597,117646,Dragonheart 2: A New Beginning,"[Action, Adventure, Comedy, Drama, Fantasy, Th...",(2000)
8900,134853,Inside Out,"[Adventure, Animation, Children, Comedy, Drama...",(2015)
9169,148775,Wizards of Waverly Place: The Movie,"[Adventure, Children, Comedy, Drama, Fantasy, ...",(2009)


## Collaborative Filtering

تا این جا ما بر اساس محتوای فیلم هایی که کاربر دیده و ژانر هایی که دوست داشته بهش فیلم پیشنهاد دادیم و حالا میخواهیم بریم سراغ روشی دیگر که بر اساس شباهت کاربر به کاربرهای دیگر و فیلم هایی که اونها دیدند و کاربرما ندیده توصیه میشه، پس به دیتاست دیگری که کاربر های دیگر رو نشون میده نیاز داریم

In [294]:
rating_data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [295]:
rating_data.drop("timestamp" , axis=1 , inplace=True)

In [296]:
movie_data.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",(1995)
1,2,Jumanji,"[Adventure, Children, Fantasy]",(1995)
2,3,Grumpier Old Men,"[Comedy, Romance]",(1995)
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",(1995)
4,5,Father of the Bride Part II,[Comedy],(1995)


اینجا دیگه نیازی به ستون ژانر ها نداریم چون بر اساس شباهت کاربر ها به یکدیگر بررسی میکنیم، شباهت کاربر ها به یکدیگر به این صورته که یک کاربر چند تا فیلم رو دیده و امتیاز داده و کاربر دیگری هم همون فیلم ها رو دیده و امتیازی مشابه داده و شباهت این کاربر هارو بر اساس روش های که برای انداره گیری شباهت یا فاصله و یا روش های دیگری که در داده کاوی استفاده میشه به دست می اوریم  

In [297]:
movie_data.drop("genres" , axis=1 , inplace=True)

In [298]:
movie_data.shape

(9742, 3)

In [299]:
user_id

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


کابرهایی که حداقل یکی از فیلمهایی که کابر ما با این آیدی ها (1 و 2و 296و 1274و 1968)  دیده رو دیده باشند رو پیدا میکنیم 

In [300]:
UserSubset=rating_data[rating_data["movieId"].isin(user_id["movieId"].tolist())]

In [301]:
UserSubset.userId

0          1
16         1
320        4
422        4
516        5
        ... 
99510    609
99534    610
99552    610
99636    610
99664    610
Name: userId, Length: 784, dtype: int64

متد گروپ بای به این صورته که در اینجا از یوزر آیدی به عنوان شاخص اصلی برای گروه بندی داده ها استفاده میکنه 

In [302]:
UserSubsetGroup=UserSubset.groupby(["userId"])

In [303]:
UserSubsetGroup.get_group(500)["userId"]

79907    500
79942    500
79957    500
Name: userId, dtype: int64

In [304]:
UserSubsetGroup.group_keys

True

لازمه که گروه ها رو مرتب سازی کنیم بر اساس این کلید (کاربر هایی که تعداد فیلم های مشترک بیشتری داشتند مثلا اونایی که هر 5 تا فیلم رو دیدند رو بیاریم بالای جدول) 

In [305]:
UserSubsetGroup=sorted(UserSubsetGroup  , key= lambda x: int(len(x[1])) , reverse=True)

In [306]:
UserSubsetGroup=UserSubsetGroup[:100]

In [332]:
UserSubsetGroup

[((91,),
         userId  movieId  rating
  14121      91        1     4.0
  14122      91        2     3.0
  14173      91      296     4.5
  14316      91     1274     5.0
  14383      91     1968     3.0),
 ((177,),
         userId  movieId  rating
  24900     177        1     5.0
  24901     177        2     3.5
  24930     177      296     5.0
  25069     177     1274     2.0
  25129     177     1968     3.5),
 ((219,),
         userId  movieId  rating
  31524     219        1     3.5
  31525     219        2     2.5
  31554     219      296     4.0
  31628     219     1274     2.5
  31680     219     1968     3.0),
 ((274,),
         userId  movieId  rating
  39229     274        1     4.0
  39230     274        2     3.5
  39288     274      296     5.0
  39448     274     1274     4.0
  39549     274     1968     4.0),
 ((298,),
         userId  movieId  rating
  44535     298        1     2.0
  44536     298        2     0.5
  44555     298      296     4.5
  44620     298    

خب حالا اینجا شباهت بین 100 کاربر را با کاربر پروفایل با استفاده از ضریب همبستگی پیرسون حساب میکنیم 
که من این فرمول رو از دوره برداشتم

In [307]:
pearsonCorrelationDict = {}

for name, group in UserSubsetGroup:
    res = int(''.join(map(str, name)))
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = user_id.sort_values(by='movieId')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = user_id[user_id['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[res] = Sxy/np.sqrt(Sxx*Syy)
    
    else:
        pearsonCorrelationDict[res] = 0

In [308]:
df = pd.DataFrame.from_dict(pearsonCorrelationDict , orient='index')
df.columns=["similarity"]
df["userId"]=df.index

In [309]:
df.index=range(len(df))

In [310]:
df.head()

,similarity,userId
0,0.438529,91
1,0.000000,177
2,0.451243,219
3,0.716115,274
4,0.959271,298


حالا میخواهیم 50 تا کاربر اول که بیشتری شباهت رو به کاربر پروفایل دارند مرتب کنیم  

In [311]:
TopUser=df.sort_values(by="similarity" , ascending =False)[:50]

In [312]:
TopUser.head()

,similarity,userId
43,1.0,132
34,1.0,18
63,1.0,305
82,1.0,489
86,1.0,525


برای اینکه به اطلاعات فیلم ها برسیم اول لازمه که دیتا های جدول به دست اومده با دیتاهای امتیاز داده شده ادغام کنیم تا بتونیم نام فیلم ها رو بر اساس کلید آیدی فیلم روپیدا کنیم  

In [313]:
TopUserRating=TopUser.merge(rating_data, left_on="userId" , right_on="userId" , how="inner")

In [314]:
TopUserRating.head()

,similarity,userId,movieId,rating
0,1.0,132,1,2.0
1,1.0,132,17,3.0
2,1.0,132,29,2.0
3,1.0,132,32,3.0
4,1.0,132,34,1.5


در این مرحله شباهت کاربر ها به کاربر پروفایل رو در امتیاز هایی که کاربران به فیلم ها دادند به دست میاریم که میشه امتیاز های وزن دهی شده
یعنی احتمالا کاربر پروفایل به فیلم 17 امتیاز 3 رو میداده 

In [315]:
TopUserRating["WeightedRating"]=TopUserRating["similarity"] * TopUserRating["rating"]

In [316]:
TopUserRating.head()

,similarity,userId,movieId,rating,WeightedRating
0,1.0,132,1,2.0,2.0
1,1.0,132,17,3.0,3.0
2,1.0,132,29,2.0,2.0
3,1.0,132,32,3.0,3.0
4,1.0,132,34,1.5,1.5


دیتاهای وزن دهی شده رو باید نرمالایز کنیم نرمال کردن در اینجا یعنی مجوع امتیاز های در وزن تقسیم بر مجموع شباهت هایی که کاربر ها به کاربر پروفایل دارند 

In [317]:
TopUserRatingWeighted=TopUserRating.groupby("movieId").sum()[["similarity" , "WeightedRating"]]

In [318]:
TopUserRatingWeighted.columns=["SumSimilarity" , "SumWeighted"]

In [319]:
TopUserRatingWeighted

,SumSimilarity,SumWeighted
movieId,,
1,36.354096,133.167946
2,31.005292,94.904257
3,8.783859,26.381456
4,0.866025,1.732051
5,7.165336,19.775255
...,...,...
187593,1.000000,5.000000
187595,0.937614,3.281651
187717,0.802955,4.014775


In [333]:
Recomm_df=pd.DataFrame()
Recomm_df["WeighAverageRecom"]=TopUserRatingWeighted["SumWeighted"]/TopUserRatingWeighted["SumSimilarity"]
Recomm_df["movieId"]=Recomm_df.index

In [334]:
Recomm_df.head()

,WeighAverageRecom,movieId
movieId,,
1,3.663080,1
2,3.060905,2
3,3.003402,3
4,2.000000,4
5,2.759850,5


In [335]:
Recomm_df=Recomm_df.sort_values(by="WeighAverageRecom" , ascending=False )[:20]

In [336]:
Recomm_df

,WeighAverageRecom,movieId
movieId,,
3310,5.0,3310
7579,5.0,7579
905,5.0,905
1211,5.0,1211
140627,5.0,140627
4298,5.0,4298
152711,5.0,152711
633,5.0,633
5537,5.0,5537


In [337]:
Recomm_df["WeighAverageRecom"]

movieId
3310      5.0
7579      5.0
905       5.0
1211      5.0
140627    5.0
4298      5.0
152711    5.0
633       5.0
5537      5.0
5485      5.0
6159      5.0
1046      5.0
105197    5.0
6184      5.0
6192      5.0
105585    5.0
32582     5.0
71268     5.0
3296      5.0
3813      5.0
Name: WeighAverageRecom, dtype: float64

In [338]:
movie_data.loc[movie_data["movieId"].isin(Recomm_df["movieId"].tolist())]

,movieId,title,year
536,633,Denise Calls Up,(1995)
687,905,It Happened One Night,(1934)
799,1046,Beautiful Thing,(1996)
912,1211,"Wings of Desire (Himmel über Berlin, Der)",(1987)
2474,3296,To Sir with Love,(1967)
2484,3310,"Kid, The",(1921)
2851,3813,Interiors,(1978)
3189,4298,Rififi (Du rififi chez les hommes),(1955)
3905,5485,Tadpole,(2002)
3936,5537,Satin Rouge,(2002)


 ده تا فیلم اولی که میتونیم به کاربر پیشنهاد بدیم که بر اساس امتیاز هایی که به فیلم هایی که دیده  و شباهت به کاربرانی که رفتار شبیه به این کاربر داشته اند به دست آوردیم 